In [ ]:
import os
import sys
import xml.etree.ElementTree as ET

from cairosvg import svg2png
from PIL import Image

In [ ]:
def modify_svg_opacity(svg_path, output_path, chosen_index):
    tree = ET.parse(svg_path)
    root = tree.getroot()
    
    # Collect elements, assuming each arrow consists of two elements (line + head)
    arrow_elements = []  # This will store tuples of (line, head)
    temp_pair = []
    
    for elem in root.iter():
        if 'fill-opacity' in elem.attrib or 'stroke-opacity' in elem.attrib or 'style' in elem.attrib:
            temp_pair.append(elem)
            if len(temp_pair) == 2:  # Every two elements form a pair
                arrow_elements.append(tuple(temp_pair))
                temp_pair = []
    
    # Ensure we have 9 pairs (18 elements total)
    if len(arrow_elements) < 9:
        print(f"Error: Found only {len(arrow_elements)} arrow pairs instead of 9!")
        return
    
    # Modify opacity for both line and head together
    for i, (line, head) in enumerate(arrow_elements[:]):  # Only modify the first 9 pairs
        if i == chosen_index:
            opacity_value = '1.0'
        else:
            opacity_value = '0.1'
        
        # Apply opacity changes to line
        line.set('stroke-opacity', opacity_value)
        if 'fill-opacity' in line.attrib:
            line.set('fill-opacity', opacity_value)

        # Apply opacity changes to head
        head.set('stroke-opacity', opacity_value)
        if 'fill-opacity' in head.attrib:
            head.set('fill-opacity', opacity_value)

        # Handle inline styles (SVGs with CSS-like attributes)
        for elem in [line, head]:
            if 'style' in elem.attrib:
                styles = elem.attrib['style'].split(';')
                new_styles = []
                for style in styles:
                    if 'fill-opacity' in style:
                        new_styles.append(f'fill-opacity:{opacity_value}')
                    elif 'stroke-opacity' in style:
                        new_styles.append(f'stroke-opacity:{opacity_value}')
                    else:
                        new_styles.append(style)
                elem.attrib['style'] = ';'.join(new_styles)
    
    # Save the modified SVG
    tree.write(output_path)
    print(f"Modified SVG saved as {output_path}")

def convert_svg_to_png(svg_path, png_path, dpi):
    svg2png(url=svg_path, write_to=png_path, dpi=dpi, output_width=dpi*4, output_height=dpi*2)
    print(f"Converted {svg_path} to {png_path}")

def create_gif(png_files, gif_path, duration=500):
    images = [Image.open(png) for png in png_files]
    images[0].save(gif_path, save_all=True, append_images=images[1:], duration=duration, loop=0)
    print(f"GIF saved as {gif_path}")

In [ ]:
i = 0
fix = 0
for j in range(0, 3):
    for k in range(0, 9):
        fix = 4
        modify_svg_opacity(f"inference_{j}.drawio.svg", f"output/output_{i}.svg", chosen_index=k+fix)
        i += 1

In [ ]:
for i in range(0, 27):
    convert_svg_to_png(f"output/output_{i}.svg", f"output/output_{i}.png", 300)

In [ ]:
gif_frames = [f"output/output_{i}.png" for i in range(27)]  # Assuming PNG files are named frame_0.png to frame_8.png
create_gif(gif_frames, "output.gif", duration=500)